<a href="https://colab.research.google.com/github/BizStat/DecisionMaking/blob/main/CHAP12_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 시뮬레이션 모형

### NumPy 라이브러리 기초

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 연립방정식의 해 구하기

In [ ]:
b = np.array([2,3,1])
A = np.array([[1,1,1],[2,-1,-3],[-1,2,2]])

In [ ]:
A

In [ ]:
A[0,0]

In [ ]:
Ainv = np.linalg.inv(A)

In [ ]:
x = np.dot(Ainv,b)

In [ ]:
print(x)

In [ ]:
Ainv @ b

### 확률분포를 따르는 난수 생성

In [ ]:
np.random.seed(1)

In [ ]:
np.random.uniform(low=0,high=1,size=10)

In [ ]:
np.random.seed(1)

In [ ]:
np.random.uniform(low=0,high=1,size=10)

In [ ]:
x = np.random.uniform(low=0,high=1,size=1000)

In [ ]:
plt.hist(x, bins=20, rwidth=0.8)
#plt.title('Uniform Random Number')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
np.random.seed(1)
x = np.random.normal(loc=20, scale=10, size=1000)

In [ ]:
plt.hist(x, bins=20, rwidth=0.8)
plt.title('Normal Random Number')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
np.random.seed(1)
x = np.random.exponential(scale=2, size=1000)

In [ ]:
np.mean(x)

In [ ]:
np.std(x)

In [ ]:
plt.hist(x, bins=20, rwidth=0.8)
plt.title('Exponential Random Number')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
np.random.seed(1)
x = np.random.poisson(lam=0.5, size=1000)

In [ ]:
np.var(x)

In [ ]:
x1, y1 = np.unique(x, return_counts=True)
plt.bar(x1, y1, width=0.8)
plt.title('Poisson Random Number')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

### 대수의 법칙

In [ ]:
x = np.random.randint(low=1, high=7, size=1000)

In [ ]:
val = np.zeros(6)
for i in range(6):
  val[i] = np.sum(x==i+1)

In [ ]:
for i in range(6):
  print(f'P({i+1}) = {val[i]/1000}')

In [ ]:
sum6 = 0
pr6 = []
for i in range(len(x)):
  if x[i] == 6:
    sum6 += 1
  pr6.append(sum6/(i+1))
#  print(f'N = {i+1}, P(6) = {pr6[i]}')

In [ ]:
n_values = range(1, len(x) + 1)
plt.plot(n_values, pr6)  # Corrected to plot pr6 directly
plt.title('Probability of rolling a 6')
plt.xlabel('Number of trials')
plt.ylabel('Probability')
plt.show()

### 원주율 구하기

### 시뮬레이션을 이용한 대기행렬모형

In [ ]:
# prompt: 도착시간과 서비스시간이 있는 창구 n개인 대기행렬분석을 위한 시뮬레이션 코드

import numpy as np
import random

class Queue:
    def __init__(self):
        self.items = []

    def is_empty(self):
        return len(self.items) == 0

    def enqueue(self, item):
        self.items.append(item)

    def dequeue(self):
        if not self.is_empty():
            return self.items.pop(0)
        else:
            return None

    def size(self):
        return len(self.items)

def simulate_queue(num_counters, arrival_rate, service_rate, simulation_time):
    # Initialize counters with arrival and service times
    counters = []
    for _ in range(num_counters):
      counters.append({'arrival_time': 0, 'service_time': 0})

    # Initialize the queue
    queue = Queue()

    # Initialize simulation parameters
    current_time = 0
    customers_served = 0
    total_waiting_time = 0

    # Generate arrival times
    arrival_times = []
    while current_time <= simulation_time:
      interarrival_time = np.random.exponential(1.0/arrival_rate)
      current_time += interarrival_time
      arrival_times.append(current_time)

    customer_index = 0

    # Run the simulation
    while customer_index < len(arrival_times) or not queue.is_empty():
        # Check for new arrivals
        while customer_index < len(arrival_times) and arrival_times[customer_index] <= current_time :
            queue.enqueue((arrival_times[customer_index], np.random.exponential(1.0/service_rate)))
            customer_index += 1

        # Check for available counters
        available_counters = []
        for i in range(num_counters):
          if counters[i]['arrival_time'] <= current_time:
            available_counters.append(i)

        # Serve customers from the queue
        for counter_index in available_counters:
          if not queue.is_empty():
            arrival, service = queue.dequeue()
            waiting_time = max(0, current_time - arrival)
            total_waiting_time += waiting_time
            customers_served += 1
            counters[counter_index]['arrival_time'] = current_time + service
            counters[counter_index]['service_time'] = service
            current_time = min(current_time + service, min(counters, key=lambda x: x['arrival_time'])['arrival_time'])
          else:
            counters[counter_index]['arrival_time'] = simulation_time + 1

        # Increment simulation time
        # current_time += 1

        current_time = min(simulation_time + 1, min(counters, key=lambda x: x['arrival_time'])['arrival_time'])

    # Calculate average waiting time
    if customers_served > 0:
        avg_waiting_time = total_waiting_time / customers_served
    else:
        avg_waiting_time = 0

    return avg_waiting_time

# Example usage
num_counters = 3
arrival_rate = 5  # Customers per time unit
service_rate = 6 # Customers per time unit
simulation_time = 100  # Time units

avg_wait = simulate_queue(num_counters, arrival_rate, service_rate, simulation_time)
print(f"Average waiting time: {avg_wait}")

### 대기행렬모형

In [ ]:
import numpy as np

def simulate_queue(simulation_time, arrival_rate, service_rate):
    """
    M/M/1 대기열 시뮬레이션
    - simulation_time: 총 시뮬레이션 시간 (분)
    - arrival_rate: 도착률 (분당 차량 수)
    - service_rate: 서비스율 (분당 차량 수)
    """
    # 랜덤 시드 설정 (재현 가능성을 위해)
    np.random.seed(42)

    # 차량 도착 시간 및 서비스 시간 생성
    arrival_times = np.cumsum(np.random.exponential(1 / arrival_rate, size=10000))
    service_times = np.random.exponential(1 / service_rate, size=10000)

    # 시뮬레이션 초기화
    current_time = 0
    queue = []  # 대기열
    departures = []  # 출발 시간 기록
    waiting_times = []  # 대기 시간 기록

    for i in range(len(arrival_times)):
        if arrival_times[i] > simulation_time:  # 시뮬레이션 시간 초과 시 종료
            break

        arrival_time = arrival_times[i]
        service_time = service_times[i]

        # 현재 시간 업데이트
        if queue:
            current_time = max(current_time, arrival_time)
        else:
            current_time = arrival_time

        # 대기 및 서비스 처리
        if current_time > arrival_time:
            waiting_time = current_time - arrival_time
            waiting_times.append(waiting_time)
            current_time += service_time
        else:
            waiting_times.append(0)  # 대기 없음
            current_time = arrival_time + service_time

        # 출발 시간 기록
        departures.append(current_time)
        queue.append(arrival_time)

    # 평균 대기 시간 및 평균 대기열 길이 계산
    average_waiting_time = np.mean(waiting_times)
    average_queue_length = np.mean([len([t for t in queue if t <= time]) for time in departures])

    return average_waiting_time, average_queue_length


# 파라미터 설정
simulation_time = 10000  # 시뮬레이션 시간 (분)
arrival_rate = 15 / 60   # 한 시간에 15대 -> 분당 0.25대
service_rate = 1 / 4     # 한 대당 서비스 시간 4분 -> 분당 0.25대

# 시뮬레이션 실행
average_waiting_time, average_queue_length = simulate_queue(simulation_time, arrival_rate, service_rate)

# 결과 출력
print(f"평균 대기시간: {average_waiting_time:.2f}분")
print(f"평균 대기열 길이: {average_queue_length:.2f}대")
